<!--<img src="images/Apache_Spark_logo.png" width="30%" align="center">-->
<img src="images/intro-logo-spark-spa.png" width="30%" align="left">

En este notebook presentaremos los conceptos básicos de Spark y su uso desde Python.

## Marco de trabajo

#### Hasta ahora:
Las tareas de "data science" y análisis de datos se han llevado a cabo “a pequeña escala”, en R/Python/MATLAB, etc.


#### Hoy en día:
Los conjuntos de datos ya no caben en memoria, así que...

* Estos lenguajes/frameworks no nos permiten escalar. 

* Hay que reimplementarlo todo en algún otro lenguaje o sistema.


#### Así que:

* La industria se está moviendo hacia una Inteligencia Empresarial basada en la toma de decisiones orientada a los datos, que se apoya en enormes conjuntos de datos.

* Principales retos: tratamiento de datos no estructurados, Big Data, datos en streaming, necesidad de HPC (*High Performance Computing*)

* Principales herramientas: paradigma *MapReduce*, *Hadoop*... y, en los últimos años, *Spark*

## ¿Qué es Spark?


* Motor multi-lenguaje, aunque implementado en **Scala**, para la ejecución de procesos de Machine Learning y Data Science en máquinas de un sólo nodo o en clusters de forma *(casi)* transparente para el desarrollador.

* Una de las herramientas más utilizadas en los últimos años en el ámbito del Big Data, desbancando a Hadoop o añadiéndole una capa de abstracción por encima.

* La API de Spark guarda una relación casi de 1-a-1 con las colecciones más comunes de lenguajes de programación de alto nivel (sobre todo de *Scala* y *Java*)... pero ¡distribuidas!

* Manejo de datos a través de estructuras de datos conocidas de otros lenguajes y librerías (¿os suenan *R*, *Pandas*,...?): tendremos **Dataframes**, pero... distribuidos o, mejor "distribuibles".

## ¿Qué es Scala?

* Lenguaje Funcional Orientado a Objetos, basado en la JVM, publicado en 2003 (por aquél entonces Java estaba en su versión 4), diseñado, entre otros, por *Martin Odersky*, también diseñador del compilador GJ, que dio lugar a **javac**.

* Comparte muchas similitudes a nivel sintáctico con Java.

* A partir de su versión 6, Java incorporó numerosas ideas de Scala.

* Scala simplifica **mucho** la sintaxis de Java, eliminando bastante *Syntactic Sugar* y con una inferencia de tipos bastante potente.

* Mecanismo de **pattern matching** para implementar instrucciones tipo *switch* más sofisticadas y extensibles, sobre todo a nivel de clases.

* Existen discusiones sobre si el futuro de Java será Scala o Kotlin, en términos de qué características serían deseables que incluyera Java en futuras versiones del lenguaje.

* Al ser funcional, son conceptos muy importantes: la **inmutabilidad** de las colecciones sobre la mutabilidad, la **recursión** sobre la iteración...

* Gracias a estas características, este lenguaje es muy apropiado para su uso en entornos **Big Data** y **Data Science**.

* Tiene documentación y tutoriales online bastante completos.

* Pero... **¡¡Es funcional!!**


<br><br><br>
<img src="images/spark_vs_hadoop.png" width="30%" align="left">
<br>




* Hadoop es una implementación `open source` del MapReduce de Google.


* Spark está implementado en Scala, un lenguaje funcional orientado a objetos, basado en la JVM.


* Ambos ofrecen una API simple para operaciones map y reduce sobre conjuntos de datos distribuidos.


* Tolerancia a fallos: entre cada operación map y reduce, se escriben datos intermedios para ser capaz de recuperarse de fallos.


* La tolerancia a fallos de Spark es mucho más eficiente que en Hadoop porque:
    - Mantiene todos los datos inmutables y en memoria
    - Las operaciones son transformaciones funcionales
    - Tolerancia a fallos: volver a aplicar las transformaciones a los datos originales


* Spark es compatible con HDFS (Hadoop Distributed FileSystem)
<br><br>



# Conceptos principales en Spark

* Spark Session: una conexión a la API de Spark


* Estructura Hardware:
    - Cluster de master + workers
    - Workflow: shuffling


* Estructuras de datos lógicas:
    - RDDs
    - PairRDDs
    
    
* Operaciones básicas:
    - Transformaciones
    - Acciones
    
    
* Librerías interesantes:
    - Spark SQL: DataFrames (y Datasets)
    - Spark Streaming API
    - MlLib
    - GraphX
    - ...

# Spark Session

### Conexión al cluster de Spark. 

Normalmente le "hablaremos" al nodo `máster` del cluster, y este le envía las tareas a los nodos `worker`.

`SparkSession` es el objeto que usaremos para llevar a cabo las operaciones de configuración y entrada contra el cluster.


In [1]:
from pyspark.sql import *

spark = SparkSession.builder.master("local").appName("NOMBRE").getOrCreate()

<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important; text-align:left;">
            <ul>
                <li>Las versiones anteriores de Spark usan <strong>SparkContext</strong> en lugar de <strong>SparkSession</strong></li>
                <li><strong>SparkContext</strong> todavía se usa, pero es transparente al desarrollador</li>
                <li>Se puede acceder a <strong>SparkContext</strong> a través de <strong>SparkSession</strong>: <strong>spark.sparkContext</strong></li>
            </ul>
        </td>
    </tr>
</table>

### Spark Hello World!


In [2]:
rdd = spark.sparkContext.parallelize(range(1,1001))
rdd.reduce(lambda x, y: x + y)

500500

Como ejemplo nos vale, pero en realidad ya tenemos una función para sumar:

In [3]:
rdd.sum()

500500

# Estructura Hardware

<img src="images/spark_structure.png" width="70%"/>

## Workflow

* El nodo 'master' distribuye los datos en bloques sobre los nodos 'worker', envía las tareas e integra los resultados de la computación.


* Los nodos 'worker' reciben los fragmentos de datos y las tareas y llevan a cabo las transformaciones y acciones sobre sus bloques de datos.


* Cada vez que nuestro proceso requiere todo el conjunto de datos para llevar a cabo una acción, el nodo 'master' recupera los bloques de datos de los 'workers', y reconstruye los datos en memoria.





<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/warning.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>Cuando los datos viajan a través de la red, se denomina <em>shuffle</em> y es <strong>realmente costoso</strong></li>
                <li>Debemos minimizar el número de veces que se requiere un <em>shuffle</em> en nuestra aplicación.</li>
                <li>Pero vamos a tomarlo con calma por ahora: necesitamos saber más conceptos relacionados con Spark para estudiar con mayor profundidad este tema.</li>
            </ul>
        </td>
    </tr>
</table>